#Primer Modelo - Clasificacion One Vs All

##Importacion de Librerias

In [171]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [172]:
# utilizado para manejos de directorios y rutas
import os
#utilizado para importar y ordenar los datos del DATASET
import pandas as pd
# Computacion vectorial y cientifica para python
import numpy as np

# Modulo de optimizacion en scipy
from scipy import optimize

from sklearn.model_selection import train_test_split

##Analisis del Dataset

In [173]:
dataframe = pd.read_csv('/content/drive/MyDrive/Primer Parcial - IA/Datasets/bank-direct-marketing-campaigns.csv')
print(dataframe.sample(n=200))

       age          job  marital            education  default housing loan  \
21244   47       admin.  married    university.degree       no      no  yes   
32682   39  blue-collar   single             basic.9y       no     yes   no   
14616   33   unemployed   single              unknown       no      no   no   
17577   38       admin.  married    university.degree       no     yes   no   
37506   64      unknown  married              unknown       no     yes  yes   
...    ...          ...      ...                  ...      ...     ...  ...   
19682   34       admin.   single    university.degree       no     yes   no   
12204   43   technician   single  professional.course       no      no   no   
1497    39  blue-collar  married             basic.9y  unknown      no   no   
33624   26      student   single          high.school       no      no   no   
33336   30       admin.  married          high.school       no     yes   no   

         contact month day_of_week  campaign  pdays

###Cambio de texto a numeros

####job

In [174]:
trabajos_unicos = dataframe['job'].unique() #Obtenemos todos los valores unicos de la columna 'job'

#Mostramos la lista de trabajos unicos y la cantidad de estos
print(trabajos_unicos)
print(len(trabajos_unicos))

['housemaid' 'services' 'admin.' 'blue-collar' 'technician' 'retired'
 'management' 'unemployed' 'self-employed' 'unknown' 'entrepreneur'
 'student']
12


In [175]:
#Se asigna valores numericos a cada uno de los trabajos
job_ordinal_map = {
    'housemaid': 1,
    'services': 2,
    'admin.': 3,
    'blue-collar': 4,
    'technician': 5,
    'retired': 6,
    'management': 7,
    'unemployed': 8,
    'self-employed': 9,
    'unknown': 10,
    'entrepreneur': 11,
    'student': 12
}

# Se aplica el mapeo a la columna 'job' y se crea una nueva columna 'job_ordinal' con los valores numericos respectivos
dataframe['job_ordinal'] = dataframe['job'].map(job_ordinal_map)

#Quitamos la columna 'job'
dataframe.drop('job', axis=1, inplace=True)

#verificamos
print(dataframe.head())

   age  marital    education  default housing loan    contact month  \
0   56  married     basic.4y       no      no   no  telephone   may   
1   57  married  high.school  unknown      no   no  telephone   may   
2   37  married  high.school       no     yes   no  telephone   may   
3   40  married     basic.6y       no      no   no  telephone   may   
4   56  married  high.school       no      no  yes  telephone   may   

  day_of_week  campaign  pdays  previous     poutcome  emp.var.rate  \
0         mon         1    999         0  nonexistent           1.1   
1         mon         1    999         0  nonexistent           1.1   
2         mon         1    999         0  nonexistent           1.1   
3         mon         1    999         0  nonexistent           1.1   
4         mon         1    999         0  nonexistent           1.1   

   cons.price.idx  cons.conf.idx  euribor3m  nr.employed   y  job_ordinal  
0          93.994          -36.4      4.857       5191.0  no          

####marital

In [176]:
marital_unicos = dataframe['marital'].unique() #Obtenemos todos los valores unicos de la columna 'marital'

#Mostramos la lista de valores unicos y la cantidad de estos
print(marital_unicos)
print(len(marital_unicos))

['married' 'single' 'divorced' 'unknown']
4


In [177]:
#Se asigna valores numericos a cada uno de los valores unicos
marital_ordinal_map = {
    'married': 1,
    'single': 2,
    'divorced': 3,
    'unknown': 4
}

# Se aplica el mapeo a la columna 'marital' y se crea una nueva columna 'marital_ordinal' con los valores numericos respectivos
dataframe['marital_ordinal'] = dataframe['marital'].map(marital_ordinal_map)

#Quitamos la columna 'marital'
dataframe.drop('marital', axis=1, inplace=True)

#verificamos
print(dataframe.head())

   age    education  default housing loan    contact month day_of_week  \
0   56     basic.4y       no      no   no  telephone   may         mon   
1   57  high.school  unknown      no   no  telephone   may         mon   
2   37  high.school       no     yes   no  telephone   may         mon   
3   40     basic.6y       no      no   no  telephone   may         mon   
4   56  high.school       no      no  yes  telephone   may         mon   

   campaign  pdays  previous     poutcome  emp.var.rate  cons.price.idx  \
0         1    999         0  nonexistent           1.1          93.994   
1         1    999         0  nonexistent           1.1          93.994   
2         1    999         0  nonexistent           1.1          93.994   
3         1    999         0  nonexistent           1.1          93.994   
4         1    999         0  nonexistent           1.1          93.994   

   cons.conf.idx  euribor3m  nr.employed   y  job_ordinal  marital_ordinal  
0          -36.4      4.857

####education

In [178]:
education_unicos = dataframe['education'].unique() #Obtenemos todos los valores unicos de la columna 'education'

#Mostramos la lista de valores unicos y la cantidad de estos
print(education_unicos)
print(len(education_unicos))

['basic.4y' 'high.school' 'basic.6y' 'basic.9y' 'professional.course'
 'unknown' 'university.degree' 'illiterate']
8


In [179]:
#Se asigna valores numericos a cada uno de los valores unicos
education_ordinal_map = {
    'basic.4y': 1,
    'high.school': 2,
    'basic.6y': 3,
    'basic.9y': 4,
    'professional.course': 5,
    'unknown': 6,
    'university.degree': 7,
    'illiterate': 8
}

# Se aplica el mapeo a la columna 'education' y se crea una nueva columna 'education_ordinal' con los valores numericos respectivos
dataframe['education_ordinal'] = dataframe['education'].map(education_ordinal_map)

#Quitamos la columna 'education'
dataframe.drop('education', axis=1, inplace=True)

#verificamos
print(dataframe.head())

   age  default housing loan    contact month day_of_week  campaign  pdays  \
0   56       no      no   no  telephone   may         mon         1    999   
1   57  unknown      no   no  telephone   may         mon         1    999   
2   37       no     yes   no  telephone   may         mon         1    999   
3   40       no      no   no  telephone   may         mon         1    999   
4   56       no      no  yes  telephone   may         mon         1    999   

   previous     poutcome  emp.var.rate  cons.price.idx  cons.conf.idx  \
0         0  nonexistent           1.1          93.994          -36.4   
1         0  nonexistent           1.1          93.994          -36.4   
2         0  nonexistent           1.1          93.994          -36.4   
3         0  nonexistent           1.1          93.994          -36.4   
4         0  nonexistent           1.1          93.994          -36.4   

   euribor3m  nr.employed   y  job_ordinal  marital_ordinal  education_ordinal  
0      4.85

####default

In [180]:
default_unicos = dataframe['default'].unique() #Obtenemos todos los valores unicos de la columna 'default'

#Mostramos la lista de valores unicos y la cantidad de estos
print(default_unicos)
print(len(default_unicos))

['no' 'unknown' 'yes']
3


In [181]:
#Se asigna valores numericos a cada uno de los valores unicos
default_ordinal_map = {
    'no': 1,
    'unknown': 2,
    'yes': 3
}

# Se aplica el mapeo a la columna 'default' y se crea una nueva columna 'default_ordinal' con los valores numericos respectivos
dataframe['default_ordinal'] = dataframe['default'].map(default_ordinal_map)

#Quitamos la columna 'default'
dataframe.drop('default', axis=1, inplace=True)

#verificamos
print(dataframe.head())

   age housing loan    contact month day_of_week  campaign  pdays  previous  \
0   56      no   no  telephone   may         mon         1    999         0   
1   57      no   no  telephone   may         mon         1    999         0   
2   37     yes   no  telephone   may         mon         1    999         0   
3   40      no   no  telephone   may         mon         1    999         0   
4   56      no  yes  telephone   may         mon         1    999         0   

      poutcome  emp.var.rate  cons.price.idx  cons.conf.idx  euribor3m  \
0  nonexistent           1.1          93.994          -36.4      4.857   
1  nonexistent           1.1          93.994          -36.4      4.857   
2  nonexistent           1.1          93.994          -36.4      4.857   
3  nonexistent           1.1          93.994          -36.4      4.857   
4  nonexistent           1.1          93.994          -36.4      4.857   

   nr.employed   y  job_ordinal  marital_ordinal  education_ordinal  \
0       5

####housing

In [182]:
y_unicos = dataframe['housing'].unique() #Obtenemos todos los valores unicos de la columna 'housing'

#Mostramos la lista de valores unicos y la cantidad de estos
print(y_unicos)
print(len(y_unicos))

['no' 'yes' 'unknown']
3


In [183]:
#Se asigna valores numericos a cada uno de los valores unicos
poutcome_ordinal_map = {
    'no': 1,
    'unknown': 2,
    'yes': 3
}

# Se aplica el mapeo a la columna 'housing' y se crea una nueva columna 'housing_ordinal' con los valores numericos respectivos
dataframe['housing_ordinal'] = dataframe['housing'].map(poutcome_ordinal_map)

#Quitamos la columna 'housing'
dataframe.drop('housing', axis=1, inplace=True)

#verificamos
print(dataframe.head())

   age loan    contact month day_of_week  campaign  pdays  previous  \
0   56   no  telephone   may         mon         1    999         0   
1   57   no  telephone   may         mon         1    999         0   
2   37   no  telephone   may         mon         1    999         0   
3   40   no  telephone   may         mon         1    999         0   
4   56  yes  telephone   may         mon         1    999         0   

      poutcome  emp.var.rate  cons.price.idx  cons.conf.idx  euribor3m  \
0  nonexistent           1.1          93.994          -36.4      4.857   
1  nonexistent           1.1          93.994          -36.4      4.857   
2  nonexistent           1.1          93.994          -36.4      4.857   
3  nonexistent           1.1          93.994          -36.4      4.857   
4  nonexistent           1.1          93.994          -36.4      4.857   

   nr.employed   y  job_ordinal  marital_ordinal  education_ordinal  \
0       5191.0  no            1                1         

####loan

In [184]:
y_unicos = dataframe['loan'].unique() #Obtenemos todos los valores unicos de la columna 'loan'

#Mostramos la lista de valores unicos y la cantidad de estos
print(y_unicos)
print(len(y_unicos))

['no' 'yes' 'unknown']
3


In [185]:
#Se asigna valores numericos a cada uno de los valores unicos
poutcome_ordinal_map = {
    'no': 1,
    'unknown': 2,
    'yes': 3
}

# Se aplica el mapeo a la columna 'loan' y se crea una nueva columna 'loan_ordinal' con los valores numericos respectivos
dataframe['loan_ordinal'] = dataframe['loan'].map(poutcome_ordinal_map)

#Quitamos la columna 'loan'
dataframe.drop('loan', axis=1, inplace=True)

#verificamos
print(dataframe.head())

   age    contact month day_of_week  campaign  pdays  previous     poutcome  \
0   56  telephone   may         mon         1    999         0  nonexistent   
1   57  telephone   may         mon         1    999         0  nonexistent   
2   37  telephone   may         mon         1    999         0  nonexistent   
3   40  telephone   may         mon         1    999         0  nonexistent   
4   56  telephone   may         mon         1    999         0  nonexistent   

   emp.var.rate  cons.price.idx  cons.conf.idx  euribor3m  nr.employed   y  \
0           1.1          93.994          -36.4      4.857       5191.0  no   
1           1.1          93.994          -36.4      4.857       5191.0  no   
2           1.1          93.994          -36.4      4.857       5191.0  no   
3           1.1          93.994          -36.4      4.857       5191.0  no   
4           1.1          93.994          -36.4      4.857       5191.0  no   

   job_ordinal  marital_ordinal  education_ordinal  defa

####contact

In [186]:
y_unicos = dataframe['contact'].unique() #Obtenemos todos los valores unicos de la columna 'contact'

#Mostramos la lista de valores unicos y la cantidad de estos
print(y_unicos)
print(len(y_unicos))

['telephone' 'cellular']
2


In [187]:
#Se asigna valores numericos a cada uno de los valores unicos
poutcome_ordinal_map = {
    'telephone': 1,
    'cellular': 2
}

# Se aplica el mapeo a la columna 'contact' y se crea una nueva columna 'contact_ordinal' con los valores numericos respectivos
dataframe['contact_ordinal'] = dataframe['contact'].map(poutcome_ordinal_map)

#Quitamos la columna 'contact'
dataframe.drop('contact', axis=1, inplace=True)

#verificamos
print(dataframe.head())

   age month day_of_week  campaign  pdays  previous     poutcome  \
0   56   may         mon         1    999         0  nonexistent   
1   57   may         mon         1    999         0  nonexistent   
2   37   may         mon         1    999         0  nonexistent   
3   40   may         mon         1    999         0  nonexistent   
4   56   may         mon         1    999         0  nonexistent   

   emp.var.rate  cons.price.idx  cons.conf.idx  euribor3m  nr.employed   y  \
0           1.1          93.994          -36.4      4.857       5191.0  no   
1           1.1          93.994          -36.4      4.857       5191.0  no   
2           1.1          93.994          -36.4      4.857       5191.0  no   
3           1.1          93.994          -36.4      4.857       5191.0  no   
4           1.1          93.994          -36.4      4.857       5191.0  no   

   job_ordinal  marital_ordinal  education_ordinal  default_ordinal  \
0            1                1                  1 

####month

In [188]:
y_unicos = dataframe['month'].unique() #Obtenemos todos los valores unicos de la columna 'month'

#Mostramos la lista de valores unicos y la cantidad de estos
print(y_unicos)
print(len(y_unicos))

['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar' 'apr' 'sep']
10


In [189]:
#Se asigna valores numericos a cada uno de los valores unicos
poutcome_ordinal_map = {
    'may': 1,
    'jun': 2,
    'jul': 3,
    'aug': 4,
    'oct': 5,
    'nov': 6,
    'dec': 7,
    'mar': 8,
    'apr': 9,
    'sep': 10
}

# Se aplica el mapeo a la columna 'month' y se crea una nueva columna 'month_ordinal' con los valores numericos respectivos
dataframe['month_ordinal'] = dataframe['month'].map(poutcome_ordinal_map)

#Quitamos la columna 'month'
dataframe.drop('month', axis=1, inplace=True)

#verificamos
print(dataframe.head())

   age day_of_week  campaign  pdays  previous     poutcome  emp.var.rate  \
0   56         mon         1    999         0  nonexistent           1.1   
1   57         mon         1    999         0  nonexistent           1.1   
2   37         mon         1    999         0  nonexistent           1.1   
3   40         mon         1    999         0  nonexistent           1.1   
4   56         mon         1    999         0  nonexistent           1.1   

   cons.price.idx  cons.conf.idx  euribor3m  nr.employed   y  job_ordinal  \
0          93.994          -36.4      4.857       5191.0  no            1   
1          93.994          -36.4      4.857       5191.0  no            2   
2          93.994          -36.4      4.857       5191.0  no            2   
3          93.994          -36.4      4.857       5191.0  no            3   
4          93.994          -36.4      4.857       5191.0  no            2   

   marital_ordinal  education_ordinal  default_ordinal  housing_ordinal  \
0    

####day_of_week

In [190]:
y_unicos = dataframe['day_of_week'].unique() #Obtenemos todos los valores unicos de la columna 'day_of_week'

#Mostramos la lista de valores unicos y la cantidad de estos
print(y_unicos)
print(len(y_unicos))

['mon' 'tue' 'wed' 'thu' 'fri']
5


In [191]:
#Se asigna valores numericos a cada uno de los valores unicos
poutcome_ordinal_map = {
    'mon': 1,
    'tue': 2,
    'wed': 3,
    'thu': 4,
    'fri': 5
}

# Se aplica el mapeo a la columna 'day_of_week' y se crea una nueva columna 'day_of_week_ordinal' con los valores numericos respectivos
dataframe['day_of_week_ordinal'] = dataframe['day_of_week'].map(poutcome_ordinal_map)

#Quitamos la columna 'day_of_week'
dataframe.drop('day_of_week', axis=1, inplace=True)

#verificamos
print(dataframe.head())

   age  campaign  pdays  previous     poutcome  emp.var.rate  cons.price.idx  \
0   56         1    999         0  nonexistent           1.1          93.994   
1   57         1    999         0  nonexistent           1.1          93.994   
2   37         1    999         0  nonexistent           1.1          93.994   
3   40         1    999         0  nonexistent           1.1          93.994   
4   56         1    999         0  nonexistent           1.1          93.994   

   cons.conf.idx  euribor3m  nr.employed   y  job_ordinal  marital_ordinal  \
0          -36.4      4.857       5191.0  no            1                1   
1          -36.4      4.857       5191.0  no            2                1   
2          -36.4      4.857       5191.0  no            2                1   
3          -36.4      4.857       5191.0  no            3                1   
4          -36.4      4.857       5191.0  no            2                1   

   education_ordinal  default_ordinal  housing_ord

####poutcome

In [192]:
y_unicos = dataframe['poutcome'].unique() #Obtenemos todos los valores unicos de la columna 'poutcome'

#Mostramos la lista de valores unicos y la cantidad de estos
print(y_unicos)
print(len(y_unicos))

['nonexistent' 'failure' 'success']
3


In [193]:
#Se asigna valores numericos a cada uno de los valores unicos
poutcome_ordinal_map = {
    'nonexistent': 1,
    'failure': 2,
    'success': 3
}

# Se aplica el mapeo a la columna 'poutcome' y se crea una nueva columna 'poutcome_ordinal' con los valores numericos respectivos
dataframe['poutcome_ordinal'] = dataframe['poutcome'].map(poutcome_ordinal_map)

#Quitamos la columna 'poutcome'
dataframe.drop('poutcome', axis=1, inplace=True)

#verificamos
print(dataframe.head())

   age  campaign  pdays  previous  emp.var.rate  cons.price.idx  \
0   56         1    999         0           1.1          93.994   
1   57         1    999         0           1.1          93.994   
2   37         1    999         0           1.1          93.994   
3   40         1    999         0           1.1          93.994   
4   56         1    999         0           1.1          93.994   

   cons.conf.idx  euribor3m  nr.employed   y  job_ordinal  marital_ordinal  \
0          -36.4      4.857       5191.0  no            1                1   
1          -36.4      4.857       5191.0  no            2                1   
2          -36.4      4.857       5191.0  no            2                1   
3          -36.4      4.857       5191.0  no            3                1   
4          -36.4      4.857       5191.0  no            2                1   

   education_ordinal  default_ordinal  housing_ordinal  loan_ordinal  \
0                  1                1                1  

####y

In [194]:
y_unicos = dataframe['y'].unique() #Obtenemos todos los valores unicos de la columna 'y'

#Mostramos la lista de valores unicos y la cantidad de estos
print(y_unicos)
print(len(y_unicos))

['no' 'yes']
2


In [195]:
#Se asigna valores numericos a cada uno de los valores unicos
y_ordinal_map = {
    'no': 1,
    'yes': 2
}

# Se aplica el mapeo a la columna 'y' y se crea una nueva columna 'y_ordinal' con los valores numericos respectivos
dataframe['y_ordinal'] = dataframe['y'].map(y_ordinal_map)

#Quitamos la columna 'y'
dataframe.drop('y', axis=1, inplace=True)

#verificamos
print(dataframe.head())

   age  campaign  pdays  previous  emp.var.rate  cons.price.idx  \
0   56         1    999         0           1.1          93.994   
1   57         1    999         0           1.1          93.994   
2   37         1    999         0           1.1          93.994   
3   40         1    999         0           1.1          93.994   
4   56         1    999         0           1.1          93.994   

   cons.conf.idx  euribor3m  nr.employed  job_ordinal  marital_ordinal  \
0          -36.4      4.857       5191.0            1                1   
1          -36.4      4.857       5191.0            2                1   
2          -36.4      4.857       5191.0            2                1   
3          -36.4      4.857       5191.0            3                1   
4          -36.4      4.857       5191.0            2                1   

   education_ordinal  default_ordinal  housing_ordinal  loan_ordinal  \
0                  1                1                1             1   
1       

###Correlacion

In [196]:
correlation_matrix = dataframe.corr()

# Imprime la matriz de correlación
print(correlation_matrix)

                          age  campaign     pdays  previous  emp.var.rate  \
age                  1.000000  0.004594 -0.034369  0.024365     -0.000371   
campaign             0.004594  1.000000  0.052584 -0.079141      0.150754   
pdays               -0.034369  0.052584  1.000000 -0.587514      0.271004   
previous             0.024365 -0.079141 -0.587514  1.000000     -0.420489   
emp.var.rate        -0.000371  0.150754  0.271004 -0.420489      1.000000   
cons.price.idx       0.000857  0.127836  0.078889 -0.203130      0.775334   
cons.conf.idx        0.129372 -0.013733 -0.091342 -0.050936      0.196041   
euribor3m            0.010767  0.135133  0.296899 -0.454494      0.972245   
nr.employed         -0.017725  0.144095  0.372605 -0.501333      0.906970   
job_ordinal          0.030413 -0.017085 -0.051677  0.051348     -0.083007   
marital_ordinal     -0.114983  0.002269 -0.017635  0.030222     -0.048931   
education_ordinal   -0.091900 -0.001521 -0.034772  0.022826     -0.033385   

In [197]:
#Eliminamos las caracyeristicas que presentan una correlacion muy cercana a cero en relacion con nuestra variable dependiente 'job_ordinal'
dataframe.drop('marital_ordinal', axis=1, inplace=True)
dataframe.drop('housing_ordinal', axis=1, inplace=True)
dataframe.drop('loan_ordinal', axis=1, inplace=True)
dataframe.drop('day_of_week_ordinal', axis=1, inplace=True)

In [198]:
correlation_matrix2 = dataframe.corr()

# Imprime la matriz de correlación
print(correlation_matrix2)

                        age  campaign     pdays  previous  emp.var.rate  \
age                1.000000  0.004594 -0.034369  0.024365     -0.000371   
campaign           0.004594  1.000000  0.052584 -0.079141      0.150754   
pdays             -0.034369  0.052584  1.000000 -0.587514      0.271004   
previous           0.024365 -0.079141 -0.587514  1.000000     -0.420489   
emp.var.rate      -0.000371  0.150754  0.271004 -0.420489      1.000000   
cons.price.idx     0.000857  0.127836  0.078889 -0.203130      0.775334   
cons.conf.idx      0.129372 -0.013733 -0.091342 -0.050936      0.196041   
euribor3m          0.010767  0.135133  0.296899 -0.454494      0.972245   
nr.employed       -0.017725  0.144095  0.372605 -0.501333      0.906970   
job_ordinal        0.030413 -0.017085 -0.051677  0.051348     -0.083007   
education_ordinal -0.091900 -0.001521 -0.034772  0.022826     -0.033385   
default_ordinal    0.164965  0.032825  0.080062 -0.102416      0.203263   
contact_ordinal   -0.0070

Se puede observar que las caracteristicas **emp.var.rate**, **euribor3m** y **nr.employed** presentan un correlacion muy fuerte entre si, lo que puede afectar a la precision de entrenamiento del modelo. Ademas de que presentan una correlacion bastante debil con la variable dependiente **job_ordinal**. Por este motivo estas caracteristicas seran removidas del dataframe dejando solamente **nr.employed** ya que es la caracteristica que mas correlacion tiene con nuestra variable dependiente.

De la misma manera se puede observar que **poutcome_ordinal** presenta una fuerte correlacion con **pdays** y **previous**. Pero en este caso solo se quitara del dataframe **poutcome_ordinal** ya que es que menos correlacion tiene con la variable dependiente, ademas de que **pdays** y **previous** no presentan una correlacion demasiado fuerte entre si.

In [199]:
dataframe.drop('emp.var.rate', axis=1, inplace=True)
dataframe.drop('euribor3m', axis=1, inplace=True)
dataframe.drop('poutcome_ordinal', axis=1, inplace=True)

###Verificamos si hay nulos

In [200]:
print(dataframe.isnull().sum())

age                  0
campaign             0
pdays                0
previous             0
cons.price.idx       0
cons.conf.idx        0
nr.employed          0
job_ordinal          0
education_ordinal    0
default_ordinal      0
contact_ordinal      0
month_ordinal        0
y_ordinal            0
dtype: int64


##Modelo - Clasificacion One Vs All

In [201]:
y = dataframe['job_ordinal']
X = dataframe.drop(columns=['job_ordinal'])

In [202]:
print(X.head())
print(y.head())

   age  campaign  pdays  previous  cons.price.idx  cons.conf.idx  nr.employed  \
0   56         1    999         0          93.994          -36.4       5191.0   
1   57         1    999         0          93.994          -36.4       5191.0   
2   37         1    999         0          93.994          -36.4       5191.0   
3   40         1    999         0          93.994          -36.4       5191.0   
4   56         1    999         0          93.994          -36.4       5191.0   

   education_ordinal  default_ordinal  contact_ordinal  month_ordinal  \
0                  1                1                1              1   
1                  2                2                1              1   
2                  2                1                1              1   
3                  3                1                1              1   
4                  2                1                1              1   

   y_ordinal  
0          1  
1          1  
2          1  
3          1  

In [203]:
# Dividimos los datos en conjuntos para entrenamiento y para prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [204]:
print(len(X_train))
print(len(y_train))

32950
32950


Pasamos los datos del DataFrame a un array de NumPy


In [205]:
X_trainNp = X_train.values
y_trainNp= y_train.values

In [206]:
print(X_trainNp.shape[1])

12


In [207]:
# La entrada es de 13 elementos contando con x0
input_layer_size  = 13

# 12 etiquetas, de 1 a 12
num_labels = 13

#y[y == 3] = 0

In [208]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [209]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X_trainNp)

In [210]:
X = X_norm

In [211]:
def sigmoid(z):
  #devuelve la sigmoide de z
    return 1.0 / (1.0 + np.exp(-z))

In [212]:
def lrCostFunction(theta, X, y, lambda_):
    # Inicializa m con la longitud de y
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)
    #inicializa el costo y el gradiente
    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T)) #calcula las hipotesis

    temp = theta
    temp[0] = 0

    #calculamos la funcion de costo regularizada
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    #calcula el gradiente regularizado
    grad = (1 / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad

In [213]:
def oneVsAll(X, y, num_labels, lambda_):
    # inicializa m y n con la cantidad de filas y columnas de X
    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1)) #inicializa all_theta

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels): #para cada clase
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50} #cantidad de iteraciones
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [214]:
lambda_ = 0.1
all_theta = oneVsAll(X, y_trainNp, num_labels, lambda_)
print(all_theta.shape)

(13, 13)


In [215]:
print(all_theta)

[[-1.24380805e+01  3.58265724e-14 -6.47898400e-13 -1.68994656e-12
   2.00285448e-12 -2.10050762e-12 -4.07196924e-13 -2.36833446e-12
   4.53808988e-13 -9.75877296e-13  1.96108508e-12  1.70864916e-12
   1.48109685e-12]
 [-4.07514194e+00  3.68752063e-01  1.85169144e-02 -5.61972257e-02
  -5.01388167e-02  1.95460382e-01  2.43966149e-01  1.45223345e-01
  -7.54480767e-01  3.43374497e-02  1.42934894e-01  6.18927539e-02
  -4.98535232e-02]
 [-2.61175361e+00 -3.16670624e-01 -2.24858164e-02  8.91669814e-02
   4.71871171e-02  1.71036329e-02 -1.59267426e-01  5.62810090e-02
  -9.24200788e-01 -2.19676929e-02 -6.37037398e-02 -5.63690268e-02
  -2.76457903e-02]
 [-1.16506402e+00 -2.01097110e-01  4.14266343e-02 -1.12898552e-02
   5.01292538e-03  2.79672293e-02  9.20581869e-02  2.69710076e-03
   4.48801443e-01 -2.26679442e-01  1.14756464e-01  1.32361957e-02
   1.68713643e-02]
 [-1.50172971e+00 -1.35242397e-01 -5.89356329e-02  4.97913623e-02
  -3.95015860e-02 -7.97829242e-02 -3.50246962e-01  1.06972891e-01


In [216]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0]; #cantidad de ejemplos
    num_labels = all_theta.shape[0] #cantidad de clases

    p = np.zeros(m)

    # Adiciona unos a X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)#adiciona la columna de unos
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1) #Realiza las predicciones

    return p

In [217]:
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y_trainNp) * 100))

X_testNp = X_test.values
y_testNp= y_test.values

X_testNp = np.concatenate([np.ones((X_testNp.shape[0], 1)), X_testNp], axis=1)

p = np.argmax(sigmoid(X_testNp.dot(all_theta.T)), axis = 1)
print(p)

print(y_testNp)
print(np.mean(p == y_testNp)*100) #indica que tan parecidos son los elementos de la prediccion y de los que deberia salir

Precision del conjuto de entrenamiento: 35.15%
[1 1 1 ... 1 1 1]
[4 4 9 ... 4 4 5]
2.791939791211459


<ipython-input-211-eb0ed38497e1>:3: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
